## Imports

In [1]:
import pandas as pd
import geopandas as gpd
import os
from geopy.geocoders import GoogleV3
import geocoder
from shapely.geometry import Point
from textwrap import shorten

## Data read-in

In [2]:
df = pd.read_csv("Condo conversion map - Sheet1.csv")

## Data work

In [3]:
# df = df.rename(columns={'2021 vaue':'2021 value',
#                        '2024 value ':'2024 value',
#                        'percent change in initial assessment ':'percent change in initial assessment'})

In [4]:
df.columns

Index(['full_address', 'project name', 'developers', 'description',
       'story_link'],
      dtype='object')

## Article link formatter

In [5]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2024/05...
1    <a href="https://therealdeal.com/miami/2024/04...
2    <a href="https://therealdeal.com/miami/2024/02...
3    <a href="https://therealdeal.com/miami/2022/07...
Name: description_link, dtype: object

In [6]:
df = df.fillna('Not available')

In [7]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2024/05...
1    <a href="https://therealdeal.com/miami/2024/04...
2    <a href="https://therealdeal.com/miami/2024/02...
3    <a href="https://therealdeal.com/miami/2022/07...
Name: description_link, dtype: object

In [8]:
df['address_full'] = df['full_address'] + ' FL'

In [9]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [10]:
df['geocoded'] = df['address_full'].apply(geolocator.geocode, timeout=10)

In [11]:
df['point'] = df['geocoded'].apply(lambda loc: tuple(loc.point) if loc else None)

In [12]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

In [13]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

# Optionally, set the coordinate reference system (CRS) for the GeoDataFrame
gdf.set_crs(epsg=4326, inplace=True)  # EPSG 4326 is the standard WGS 84 latitude and longitude system

,full_address,project name,developers,description,story_link,short_description,remaining_desc,description_link,address_full,geocoded,point,lat,lon,altitude,geometry
0,1451 Northwest 14th Street in Miami,River District 14,Alta Development,"Henry Pino's firm plans a 16-story, 283-unit s...",https://therealdeal.com/miami/2024/05/01/henry...,Henry Pino's firm,"plans a 16-story, 283-unit short-term rental-...","<a href=""https://therealdeal.com/miami/2024/05...",1451 Northwest 14th Street in Miami FL,"(1451 NW 14th St, Miami, FL 33125, USA, (25.78...","(25.78767, -80.21982799999999, 0.0)",25.787670,-80.219828,0.0,POINT (-80.21983 25.78767)
1,4011 Salzedo Avenue in Coral Gables,Cassia,Alta Developers,"Alta Developers launched sales of Cassia, a pl...",https://therealdeal.com/miami/2024/04/25/alta-...,Alta Developers,"launched sales of Cassia, a planned 12-story,...","<a href=""https://therealdeal.com/miami/2024/04...",4011 Salzedo Avenue in Coral Gables FL,"(4011 Salzedo St, Coral Gables, FL 33146, USA,...","(25.7347112, -80.26037099999999, 0.0)",25.734711,-80.260371,0.0,POINT (-80.26037 25.73471)
2,1776 Polk Street in Hollywood,Circ Residences,GCF Development,Chip Abele's GCF Development completed the 25-...,https://therealdeal.com/miami/2024/02/08/chip-...,Chip Abele's GCF,"Development completed the 25-story, 386-unit ...","<a href=""https://therealdeal.com/miami/2024/02...",1776 Polk Street in Hollywood FL,"(1776 Polk St, Hollywood, FL 33020, USA, (26.0...","(26.0133029, -80.1423733, 0.0)",26.013303,-80.142373,0.0,POINT (-80.14237 26.01330)
3,398 Northeast Fifth Street in Miami,The Elser Hotel & Residences,PMG,PMG changed course with its Society Biscayne a...,https://therealdeal.com/miami/2022/07/06/chang...,PMG changed course,with its Society Biscayne apartment tower in ...,"<a href=""https://therealdeal.com/miami/2022/07...",398 Northeast Fifth Street in Miami FL,"(398 NE 5th St, Miami, FL 33132, USA, (25.7788...","(25.7788802, -80.1890556, 0.0)",25.778880,-80.189056,0.0,POINT (-80.18906 25.77888)


In [14]:
gdf[['geocoded','point']] = gdf[['geocoded','point']].astype(str)

In [15]:
# Export to GeoJSON
gdf.to_file("SoFlaCondoconversions_06_06_24.geojson", driver='GeoJSON')